##                      Imagen Médica              -                Máster Visión Artificial (2023-2024)

**María Cornejo Antonaya (m.cornejo.2023@alumnos.urjc.es)**

**Nuria Miralles Gavara (n.miralles.2023@alumnos.urjc.es)**

**Juan Montes Cano (juan.montes@urjc.es)**

# Práctica 1: Filtrado y Mejora de Imagen Médica
### Introducción
El objetivo de la práctica es evaluar tres algoritmos de filtrado de imagen médica junto a la creación de un mapa paramétrico de estas imágenes. El resultado esperado es la mejora de la calidad de la imagen original.

### 1. Filtrado anisotrópico por difusión
El filtrado anisotrópico por difusión es una técnica que persigue eliminar el ruido presente en las zonas homogéneas de una imagen, preservando simultáneamente los contornos que delimitan las distintas regiones. Este filtro logra una notable mejora en la calidad de la imagen al eliminar las imperfecciones sin comprometer la nitidez de la misma.
En este ejercicio se configura el filtro anisotrópico para las imágenes *T1.png* y *T2.png*. Con el objetivo de obtener resultados más evidentes, se ha añadido ruido Rayleigh a las imágenes proporcionadas. Este tipo de ruido es característico del procesamiento de imágenes de resonancia magnética (MRI).
La función de densidad de probabilidad de este ruido se define como:


$$ P(q) = q \sigma^2 e^{-\frac{1}{2} \frac{q^2}{\sigma^2}} $$

La implementación del filtro anisotrópico se ha llevado a cabo mediante el algoritmo de Perona-Malik para suavizar imágenes. El código correspondiente se encuentra disponible en el siguiente repositorio: 
[Repositorio](https://github.com/krishanuskr/ImageRestoration/blob/master/imagerestoration.py)

Este filtro está definido por un conjunto de parámetros cuya configuración permite realizar un filtrado específico para cada imagen. A continuación, se procede a analizar cada uno de estos parámetros para su posterior ajuste óptimo para ambas imágenes.
Parámetros:
- *"img"*: Imagen de entrada que se desea filtrar.
- *"K"*: Coeficiente de conductancia utilizado para medir la sensibilidad del algoritmo frente a los detalles de la imagen en ambas funciones, exponencial y Cauchy.
- *"LAMBDA"*: Parámetro cuyo valor máximo es 0.25 asegurando que la imagen resultante sea estable. Su valor cuantifica la difusión aplicada en cada iteración del algoritmo. Por lo tanto, a medida que aumenta su valor, la imagen será más suave.
- *"gfunction"*: Selección de la función de difusión a utilizar. Se distingue entre la función exponencial *(‘Exponential’)* y la función de Cauchy *(‘Cauchy’)*. La primera opción tiene mayor sensibilidad a cambios bruscos en la intensidad de la imagen proporcionando un mayor suavizado en áreas homogéneas. Por otro lado, la segunda opción es menos sensible en comparación con la función exponencial y favorece las regiones grandes sobre las pequeñas.
- *"“nIterations"*:  Número de interacciones. A medida que se aumenta su valor, la imagen de salida se suaviza más, ya que los efectos de la difusión se propagan sobre un mayor número de iteraciones. 

##### Evaluación del parámetro *K*

<img src="extremos_K.png" alt="image" width="700"/>

#### Evaluación del parámetro *$\lambda$*
<img src="extremos_lambda.png" alt="image" width="700"/>


In [ ]:

## APARTADO P1 Y P2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def PeronaMalik_Smoother(image,K,LAMBDA,gfunction,nIterations,convert_to_grayscale=True): 
    # REFERENCE: https://github.com/krishanuskr/ImageRestoration/blob/master/imagerestoration.py
    
    print ('Image shape: ', image.shape)

    
    if convert_to_grayscale == True:
        try:
            image = np.mean(image,axis=2)
        except IndexError:
            raise IndexError('Image is already 2D, so is assumed grayscale already. Check shape.')

    
    #Get number of color channels of image [1 for grayscale, 3 for RGB, 4 for RGBA]
    #Assuming the image shape is heigth x width for grayscale,
    #or heigth x width x Nchannels for color. But not expecting more than 3 dimensions.
    nChannels = 1 if image.ndim == 2 else image.shape[2]
    print ('nChannels',nChannels)
    
    #In the case of a grayscale image, to make things easier later, just make the grayscale image have a 3rd axis of length 1
    if nChannels == 1:
        image = np.expand_dims(image,axis=2)
    
    #4D Container array of all iterations of image diffusion
    image_stack = np.expand_dims(image,axis=0)
    
    #Do nIterations of diffusion:
    for i in range(nIterations):
        if i % 10 == 0:
            print ('Starting iteration {0} of {1}'.format(i,nIterations))
        image_t = np.zeros(image.shape)
        for channel in range(nChannels):
            temp = image_stack[-1][:,:,channel]
            
            #Following equation 8 in paper: calculate nearest neighbor differences to approximate gradient of image intensity
            vert_diff = np.diff(temp,axis=0)
            horiz_diff = np.diff(temp,axis=1)
            nanrow = np.expand_dims(np.nan*np.ones(vert_diff.shape[1]),axis=0)
            nancol = np.expand_dims(np.nan*np.ones(horiz_diff.shape[0]),axis=0).T
            grad_S = np.vstack((vert_diff,nanrow)) #NaN on bottom row
            grad_N = np.vstack((nanrow,-vert_diff)) #NaN on top row, and negated diffs since going opposite direction from np.diff() default
            grad_E = np.hstack((horiz_diff,nancol)) #NaN on right column
            grad_W = np.hstack((nancol,-horiz_diff)) #NaN on left column, and negated diffs since going opposite direction from np.diff() default
            
            #Following equation 10 in paper: calculate conduction coefficients
            #Technically, the coefficients should be more appropriately be evaluated at the halfway point between pixels, not at the pixels themselves.
            #But this is more complicated for approximately same results (according to authors). So use the same values for gradients as above.
            if gfunction == 'Exponential':
                c_S = np.exp(-(grad_S/K)**2)
                c_N = np.exp(-(grad_N/K)**2)
                c_E = np.exp(-(grad_E/K)**2)
                c_W = np.exp(-(grad_W/K)**2)
                
            if gfunction == 'Cauchy':
                c_S = 1./(1.+(grad_S/K)**2)
                c_N = 1./(1.+(grad_N/K)**2)
                c_E = 1./(1.+(grad_E/K)**2)
                c_W = 1./(1.+(grad_W/K)**2)
            
            #Examine the conduction coefficients:
            
            #Following equation 7 in paper: Update the image using the diffusion equation:
            temp2 = temp + LAMBDA*(c_S*grad_S + c_N*grad_N + c_E*grad_E + c_W*grad_W)
            
            #Reset boundaries since the paper uses adiabatic boundary conditions and above steps intentionally set boudnaries to NaNs
            temp2[:,0] = temp[:,0] #Left edge
            temp2[:,-1] = temp[:,-1] #Right edge
            temp2[-1,:] = temp[-1,:] #Bottom edge
            temp2[0,:] = temp[0,:] #Top edge
            
            #Update this channel of the image at this time step
            image_t[:,:,channel] = temp2

        image_t = np.expand_dims(image_t,axis=0)
        image_stack = np.append(image_stack,image_t,axis=0)    
    
    
    #image_stack is stack of all iterations.
    #iteration 0 is original image, iteration -1 is final image.
    #Intermediate images are also returned for visualization and diagnostics
    return image_stack


### MULTICHANNEL


def PeronaMalik_Smoother_MC(image,image1,K,LAMBDA,gfunction,nIterations,convert_to_grayscale=True): # REFERENCE: https://github.com/krishanuskr/ImageRestoration/blob/master/imagerestoration.py
    
    print ('Image shape: ', image.shape)

    
    if convert_to_grayscale == True:
        try:
            image = np.mean(image,axis=2)
        except IndexError:
            raise IndexError('Image is already 2D, so is assumed grayscale already. Check shape.')

    
    #Get number of color channels of image [1 for grayscale, 3 for RGB, 4 for RGBA]
    #Assuming the image shape is heigth x width for grayscale,
    #or heigth x width x Nchannels for color. But not expecting more than 3 dimensions.
    nChannels = 1 if image.ndim == 2 else image.shape[2]
    print ('nChannels',nChannels)
    
    #In the case of a grayscale image, to make things easier later, just make the grayscale image have a 3rd axis of length 1
    if nChannels == 1:
        image = np.expand_dims(image,axis=2)
        image1 = np.expand_dims(image1,axis=2)
    
    #4D Container array of all iterations of image diffusion
    image_stack = np.expand_dims(image,axis=0)
    image_stack_1 = np.expand_dims(image1,axis=0)

    #Do nIterations of diffusion:
    for i in range(nIterations):
        if i % 10 == 0:
            print ('Starting iteration {0} of {1}'.format(i,nIterations))
        image_t = np.zeros(image.shape)
        image_t1 = np.zeros(image1.shape)
        for channel in range(nChannels):
            temp = image_stack[-1][:,:,channel]
            temp1 = image_stack_1[-1][:,:,channel]

            #Following equation 8 in paper: calculate nearest neighbor differences to approximate gradient of image intensity
            vert_diff = np.diff(temp,axis=0)
            vert_diff1 = np.diff(temp1,axis=0)
            horiz_diff = np.diff(temp,axis=1)
            horiz_diff1 = np.diff(temp1,axis=1)
            nanrow = np.expand_dims(np.nan*np.ones(vert_diff.shape[1]),axis=0)
            nanrow1 = np.expand_dims(np.nan*np.ones(vert_diff1.shape[1]),axis=0)
            nancol = np.expand_dims(np.nan*np.ones(horiz_diff.shape[0]),axis=0).T
            nancol1 = np.expand_dims(np.nan*np.ones(horiz_diff1.shape[0]),axis=0).T
            grad_S = np.vstack((vert_diff,nanrow)) #NaN on bottom row
            grad_S1 = np.vstack((vert_diff1,nanrow1)) #NaN on bottom row
            grad_N = np.vstack((nanrow,-vert_diff)) #NaN on top row, and negated diffs since going opposite direction from np.diff() default
            grad_N1 = np.vstack((nanrow1,-vert_diff1)) #NaN on top row, and negated diffs since going opposite direction from np.diff() default
            grad_E = np.hstack((horiz_diff,nancol)) #NaN on right column
            grad_E1 = np.hstack((horiz_diff1,nancol1)) #NaN on right column
            grad_W = np.hstack((nancol,-horiz_diff)) #NaN on left column, and negated diffs since going opposite direction from np.diff() default
            grad_W1 = np.hstack((nancol1,-horiz_diff1)) #NaN on left column, and negated diffs since going opposite direction from np.diff() default

            #Recalculamos los gradientes como dice en el pdf
            grad_W = (grad_W**2 + grad_W1**2)**0.5
            grad_E = (grad_E**2 + grad_E1**2)**0.5
            grad_N = (grad_N**2 + grad_N1**2)**0.5
            grad_S = (grad_S**2 + grad_S1**2)**0.5
            
            #Following equation 10 in paper: calculate conduction coefficients
            #Technically, the coefficients should be more appropriately be evaluated at the halfway point between pixels, not at the pixels themselves.
            #But this is more complicated for approximately same results (according to authors). So use the same values for gradients as above.
            if gfunction == 'Exponential':
                c_S = np.exp(-(grad_S/K)**2)
                c_N = np.exp(-(grad_N/K)**2)
                c_E = np.exp(-(grad_E/K)**2)
                c_W = np.exp(-(grad_W/K)**2)
                
            if gfunction == 'Cauchy':
                c_S = 1./(1.+(grad_S/K)**2)
                c_N = 1./(1.+(grad_N/K)**2)
                c_E = 1./(1.+(grad_E/K)**2)
                c_W = 1./(1.+(grad_W/K)**2)
            
            #Examine the conduction coefficients:
            
            #Following equation 7 in paper: Update the image using the diffusion equation:
            temp2 = temp + LAMBDA*(c_S*grad_S + c_N*grad_N + c_E*grad_E + c_W*grad_W)
            temp3 = temp1 + LAMBDA*(c_S*grad_S + c_N*grad_N + c_E*grad_E + c_W*grad_W)
            #Reset boundaries since the paper uses adiabatic boundary conditions and above steps intentionally set boudnaries to NaNs
            temp2[:,0] = temp[:,0] #Left edge
            temp3[:,0] = temp1[:,0] #Left edge
            temp2[:,-1] = temp[:,-1] #Right edge
            temp3[:,-1] = temp1[:,-1] #Right edge
            temp2[-1,:] = temp[-1,:] #Bottom edge
            temp3[-1,:] = temp1[-1,:] #Bottom edge
            temp2[0,:] = temp[0,:] #Top edge
            temp3[0,:] = temp1[0,:] #Top edge
            
            #Update this channel of the image at this time step
            image_t[:,:,channel] = temp2
            image_t1[:,:,channel] = temp3

        image_t = np.expand_dims(image_t,axis=0)
        image_t1 = np.expand_dims(image_t1,axis=0)
        image_stack = np.append(image_stack,image_t,axis=0)
        image_stack_1 = np.append(image_stack_1,image_t1,axis=0)
    
    
    #image_stack is stack of all iterations.
    #iteration 0 is original image, iteration -1 is final image.
    #Intermediate images are also returned for visualization and diagnostics
    return image_stack,image_stack_1


if __name__ == '__main__':

    #Load test image
    image = mpimg.imread('Material_P1/T2.png')
    image1 = mpimg.imread('Material_P1/T1.png')
    #Set algorithm parameters
    K = [1]
    LAMBDA = [0.15]
    nIterations = [5] #100
    gfunction = 'Exponential' #'Cauchy'


    
    #plt.title('Original',fontsize=30)
    #plt.imshow(image,interpolation='None',cmap='gray')
    #plt.show()

    #Add noise to image
    noise = np.random.normal(0,.01,image.shape)
    #image = image + noise

    #Add rician noise to the image
    noise = np.random.rayleigh(0.05,image.shape)
    image = image + noise
    
    plt.title('Original + Noise',fontsize=30)
    plt.imshow(image,interpolation='None',cmap='gray')
    #plt.show()
    #Plot grayscale example
    #Plot in the same plot the images with different K and Lambda values

    # Create a 6x4 grid plot
    fig, axes = plt.subplots(1, 4, figsize=(12, 18))
    axes = axes.flatten()
    axes[0].imshow(image,interpolation='None',cmap='gray')
    axes[0].set_title('Original + Noise',fontsize=10)
    axes[1].imshow(image1,interpolation='None',cmap='gray')
    axes[1].set_title('Original + Noise',fontsize=10)

    cont = 2
    for  k in K:
        for l in LAMBDA:
            for iteraciones in nIterations:
                for funcion in ['Cauchy']:
                    
                    PMimage_stack1,PMimage_stack2 = PeronaMalik_Smoother_MC(image,image1,k,l,funcion,iteraciones,convert_to_grayscale=False)
                    #I want to save each image into the axes array
                    axes[cont].set_title('K={0}, Lambda={1}, Iter={2}, Func={3}'.format(k,l,iteraciones,funcion),fontsize=10)
                    #I want it to be grayscale
                    axes[cont].imshow(np.squeeze(PMimage_stack1[-1]),interpolation='None',cmap='gray')
                    cont+=1
                    axes[cont].set_title('K={0}, Lambda={1}, Iter={2}, Func={3}'.format(k,l,iteraciones,funcion),fontsize=10)
                    axes[cont].imshow(np.squeeze(PMimage_stack2[-1]),interpolation='None',cmap='gray')
                    axes[cont].axis('off')
                    cont+=1
                    print("ITERATION: ",cont)
    # Adjust layout
    plt.tight_layout()
    plt.show()


Partiendo del objetivo de aplicar el filtro anisotrópico para eliminar el ruido y preservar los bordes principales, dado que no se busca resaltar el detalle sino la forma, podemos concluir que se ha logrado alcanzar el objetivo principal. El filtro difumina las texturas homogéneas, eliminando el ruido Rayleigh, al mismo tiempo que conserva los bordes de las imágenes de entrada.

Parámetros del filtro anisotrópico para la imagen *“T1.png”* + Rayleigh (0.1):
*K = 15; LAMBDA = 0.03; gfunction = Exponencial; nIterations = 20.*

##### Resultado T1
<img src="resultado_T1.png" alt="image" width="700"/>

Parámetros del filtro anisotrópico para la imagen *“T2.png”* + Rayleigh (0.05):
*K = 1; LAMBDA = 0.15; gfunction = Exponencial; nIterations = 5.*

##### Resultado T2
<img src="resultado_T2.png" alt="image" width="700"/>





##### P2. Filtrado al caso de dos imágenes

Cuanto más iteraciones más grado de importancia tiene el gradiente calculado 
los bordes se mantienen
Comparando ambos resultados podemos concluir 
se centra más en la estructura de la imagen es decir respetar los bordes y las regiones lo pilla mejor pero en cambio ensucia más imagen  perdiendo los detalles
Comparando el resultado con el monocanal este respeta pero los bordes haciéndolos más nítidos mientras que en la segunda respeta mejor los bordes pero pierde la homogeneidad ganada en el caso de monocanal

##### Evaluación multicanal
<img src="multicanal.png" alt="image" width="700"/>


### 2. Filtrado no local de media (Non-local means)
En este ejercicio, se procede a implementar un algoritmo de filtrado no local de media (Non-local means) para imágenes médicas. 
[Repositorio de referencia](https://github.com/praveenVnktsh/Non-Local-Means/blob/main/main.py)

 



In [ ]:
import cv2
import numpy as np
from multiprocessing import Pool
import os
import matplotlib.pyplot as plt
import dif_aniso as da

def nonLocalMeans(noisy, params = tuple(), verbose = True): ### REFERENCIA : https://github.com/praveenVnktsh/Non-Local-Means/blob/main/main.py
  '''
  Performs the non-local-means algorithm given a noisy image.
  params is a tuple with:
  params = (bigWindowSize, smallWindowSize, h)
  Please keep bigWindowSize and smallWindowSize as even numbers
  '''

  bigWindowSize, smallWindowSize, h  = params
  padwidth = bigWindowSize//2
  image = noisy.copy()

  # The next few lines creates a padded image that reflects the border so that the big window can be accomodated through the loop
  paddedImage = np.zeros((image.shape[0] + bigWindowSize,image.shape[1] + bigWindowSize))
  paddedImage = paddedImage.astype(np.uint8)
  paddedImage[padwidth:padwidth+image.shape[0], padwidth:padwidth+image.shape[1]] = image
  paddedImage[padwidth:padwidth+image.shape[0], 0:padwidth] = np.fliplr(image[:,0:padwidth])
  paddedImage[padwidth:padwidth+image.shape[0], image.shape[1]+padwidth:image.shape[1]+2*padwidth] = np.fliplr(image[:,image.shape[1]-padwidth:image.shape[1]])
  paddedImage[0:padwidth,:] = np.flipud(paddedImage[padwidth:2*padwidth,:])
  paddedImage[padwidth+image.shape[0]:2*padwidth+image.shape[0], :] =np.flipud(paddedImage[paddedImage.shape[0] - 2*padwidth:paddedImage.shape[0] - padwidth,:])
  


  iterator = 0
  totalIterations = image.shape[1]*image.shape[0]*(bigWindowSize - smallWindowSize)**2

  if verbose:
    print("TOTAL ITERATIONS = ", totalIterations)

  outputImage = paddedImage.copy()

  smallhalfwidth = smallWindowSize//2


  # For each pixel in the actual image, find a area around the pixel that needs to be compared
  for imageX in range(padwidth, padwidth + image.shape[1]):
    for imageY in range(padwidth, padwidth + image.shape[0]):
      
      bWinX = imageX - padwidth
      bWinY = imageY - padwidth

      #comparison neighbourhood
      compNbhd = paddedImage[imageY - smallhalfwidth:imageY + smallhalfwidth + 1,imageX-smallhalfwidth:imageX+smallhalfwidth + 1]
      
      
      pixelColor = 0
      totalWeight = 0

      # For each comparison neighbourhood, search for all small windows within a large box, and compute their weights
      for sWinX in range(bWinX, bWinX + bigWindowSize - smallWindowSize, 1):
        for sWinY in range(bWinY, bWinY + bigWindowSize - smallWindowSize, 1):   
          #find the small box       
          smallNbhd = paddedImage[sWinY:sWinY+smallWindowSize + 1,sWinX:sWinX+smallWindowSize + 1]
          euclideanDistance = np.sqrt(np.sum(np.square(smallNbhd - compNbhd)))
          #weight is computed as a weighted softmax over the euclidean distances
          weight = np.exp(-euclideanDistance/h)
          totalWeight += weight
          pixelColor += weight*paddedImage[sWinY + smallhalfwidth, sWinX + smallhalfwidth]
          iterator += 1

          if verbose:
            percentComplete = iterator*100/totalIterations
            if percentComplete % 5 == 0:
              print('% COMPLETE = ', percentComplete)

      pixelColor /= totalWeight
      outputImage[imageY, imageX] = pixelColor

  return outputImage[padwidth:padwidth+image.shape[0],padwidth:padwidth+image.shape[1]]


def add_gaussian_noise(image, mean=0, std_dev=0.5):
    """
    Agrega ruido gaussiano a una imagen.
    """
    noisy_image = np.copy(image)
    h, w = image.shape
    noise = np.random.normal(mean, std_dev, (h, w)).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    return noisy_image


# Cargar la imagen
original_image = cv2.imread('Material_P1/T1.png', cv2.IMREAD_GRAYSCALE)

# Agregar ruido gaussiano a la imagen
noisy_image = add_gaussian_noise(original_image)

gParams = {
    'bigWindow' : 20,
    'smallWindow':6,
    'h':14,
}
imagen_filtrogaussiano = cv2.GaussianBlur(noisy_image, (5, 5), 0)
anisotropic_img = da.PeronaMalik_Smoother(noisy_image, 5, 0.01, "Exponential",15,False)[-1]
#perform NLM filtering
filtered_image = nonLocalMeans(noisy_image, params = (gParams['bigWindow'], gParams['smallWindow'],gParams['h']))

# Mostrar las imágenes
plt.figure(figsize=(15, 5))
#
plt.subplot(1, 5, 1)
plt.imshow(original_image, cmap='gray')
plt.title('Imagen Original')
plt.axis('off')

plt.subplot(1, 5, 2)
plt.imshow(noisy_image, cmap='gray')
plt.title('Imagen con Ruido Gaussiano')
plt.axis('off')

plt.subplot(1, 5, 3)
plt.imshow(filtered_image, cmap='gray')
plt.title('Imagen Filtrada con NLM')
plt.axis('off')

plt.subplot(1, 5, 4)
plt.imshow(anisotropic_img, cmap='gray')
plt.title('Imagen Filtrada con Perona Malik')
plt.axis('off')

plt.subplot(1, 5, 5)
plt.imshow(imagen_filtrogaussiano, cmap='gray')
plt.title('Imagen Filtrada con Filtro Gaussiano')
plt.axis('off')


plt.show()


### P3. Filtrado non-local means
El filtrado Non-local means es una técnica utilizada en el procesamiento de imágenes para reducir el ruido y mejorar la calidad de las imágenes. A diferencia de otros métodos de filtrado que se centran en la información local de cada píxel, el filtrado non-local means busca similitudes en regiones más amplias de la imagen.

Este se basa en sustituir el valor de cada píxel por la media ponderada de los píxeles en la imagen que son considerados similares a través de una métrica de similitud. Esta similitud se mide comparando vecindarios de píxeles alrededor de cada píxel. Cuanto más similar sea el vecindario de un píxel con otros vecindarios en la imagen, mayor será su contribución en el cálculo de la media para ese píxel.

Para probar el algoritmo se ha usado la imagen T1 y, al igual que en el ejercicio anterior, la imagen de entrada se ha preprocesado para añadirle un ruido de tipo gaussiano con media 0 y desviación estándar 0.5.

<img src="noised_T1.png" alt="image" width="300"/>
<img src="noised_T2.png" alt="image" width="300"/>

### P4. Comparar resultado de filtrar imagen T1 con este filtro, filtrado gaussiano y filtro de Perona-Malik

El filtrado gaussiano, implementado mediante la función cv2.GaussianBlur de OpenCV, se utiliza para desenfocar la imagen. Cuanto mayor sea la desviación estándar especificada en el núcleo gaussiano, mayor será el desenfoque aplicado a la imagen. Esto resulta en una imagen más suavizada, donde los bordes se vuelven menos definidos y la imagen se vuelve más homogénea. A medida que aumenta la desviación estándar, se pierden los detalles finos y la imagen se vuelve más borrosa.

El filtrado anisotrópico o de Perona-Malik, implementado en el ejercicio P1, es una función dependiente del gradiente que se utiliza para realizar los bordes. En este caso, no da buenos resultados apenas consigue disminuir el ruido gaussiano de la imagen.

Por otro lado, el filtrado Non-local means, o medios no locales, a diferencia del filtrado gaussiano que se centra en la información local de cada píxel, busca similitudes en regiones más amplias de la imagen. Esto permite una reducción del ruido más efectiva mientras se conservan los detalles finos y los bordes. En comparación con el filtrado gaussiano, el filtrado non-local means produce una imagen más nítida y con bordes mejor definidos, lo que da a la imagen una apariencia mejorada y más "reluciente"

<img src="P4.png" alt="image" width="900"/>


### 3. Cálculo de mapas de hierro del cerebro

En este apartado se persigue medición no invasiva de la presencia de hierro
mediante resonancia magnética. 
La imagen de resonancia magnética utiliza pulsos de radiofrecuencia
ajustados al giro de los protones, dando lugar a una velocidad de precisión
que define el movimiento circular rotatorio de los núcleos atómicos al someterse
a un campo magnético.

Esta velocidad es distinta para el hierro, ya que su entorno químico
experimenta un campo magnético diferente que provoca un desfase representado
por la constante *T2*. Este concepto resulta fundamental para la detección efectiva de 
la presencia de hierro.

Partiendo de los datos de intensidad y el tiempo de eco de cada una de las imágenes facilitadas (“Hierro_TE*tif”), se procede al cálculo de los mapas de hierro del cerebro *(T2)*.
A continuación, se muestra el análisis teórico empleado para la resolución del sistema de ecuaciones derivado de la definición de la señal en función del tiempo:

<div id="eq1">

$$s(t) = s_0 \cdot e^{-\frac{t}{T_2}}; (1)$$ 
</div>

Siendo:

*“s”*: Intensidad de los píxeles de la adquisición

 *“t”*: Tiempo de eco

*“$s_0$”*: Señal inicial 

*“$T_2$”*: Tiempo de relajación


Para resolver este sistema, se ha empleado la técnica de optimización de mínimos cuadrados, expresada por la siguiente ecuación: 
 
<div id="eq2">

$$ y = A · e^{Bx}; (2) $$
</div>

Tomando el algoritmo, se obtiene:

<div id="eq3">

$$ln(y) = ln(A) + Bx; (3)$$
</div>

Comparando con la ecuación ([1](#eq1)), se deduce:

$$A = S_0;$$
$$ B = R_2 = - 1/T_2; $$
$$ x = TE; $$
$$ y = S;$$

<div id="eq4">

$$ln(S(t)) = ln(S_0) + R_2;  (4)$$  
</div>

Teniendo en cuenta estas equivalencias y las restricciones del algoritmo, se han excluido los valores de intensidad nulos para evitar la indeterminación del logaritmo de cero. 
Posteriormente, se muestra la implementación y solución de la ecuación ([4](#eq4))

<img src="p5_imagenes.png" alt="image" width="300"/>


<img src="P5_heatmap.png" alt="image" width="300"/>


<img src="P5.png" alt="image" width="300"/>